In [1]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr # oh yeah!
import pandas as pd
import io
# imports
import os
import glob
from dotenv import load_dotenv
import gradio as gr
# imports for langchain, plotly and Chroma

from langchain.document_loaders import DirectoryLoader, TextLoader,  PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.callbacks import StdOutCallbackHandler
import gradio as gr
import openai
import pandas as pd
import numpy as np
import io
import random
import tempfile
from datetime import timedelta

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyAt


In [3]:
openai = OpenAI()

google.generativeai.configure()

In [4]:
MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [5]:
folders = glob.glob("pdf/*.pdf")

documents = []

for folder in folders:
    doc_type = os.path.splitext(os.path.basename(folder))[0]

    # Load all PDFs inside this folder
    loader = PyPDFLoader(folder)
    folder_docs = loader.load()

    # Attach folder name as metadata
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

print(f"Loaded {len(documents)} documents")

Loaded 210 documents


In [6]:

folders = glob.glob("knowledge-base/*")

text_loader_kwargs = {'encoding': 'utf-8'}

#documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [8]:
print(f"Loaded {len(documents)} documents")

Loaded 241 documents


In [9]:
# print (chunks[100])

In [10]:
embeddings = OpenAIEmbeddings()

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 333 documents


In [11]:
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


In [12]:
# Let's investigate what gets sent behind the scenes

llm = ChatOpenAI(temperature=0.5, model_name=MODEL)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, callbacks=[StdOutCallbackHandler()])

C:\Users\hp\AppData\Local\Temp\ipykernel_18772\2062993249.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [13]:
def generate_servicenow_data(n=10000):
    np.random.seed(42)
    random.seed(42)

    priorities = ["Low", "Medium", "High", "Critical"]
    impacts = ["Low", "Medium", "High"]
    urgencies = ["Low", "Medium", "High"]
    statuses = ["Open", "In Progress", "Resolved", "Closed"]
    categories = ["Network", "Software", "Hardware", "Database", "Security"]
    subcategories = {
        "Network": ["VPN", "LAN", "WAN"],
        "Software": ["Email", "OS", "Application"],
        "Hardware": ["Laptop", "Desktop", "Printer"],
        "Database": ["Oracle", "MySQL", "SQL Server"],
        "Security": ["Phishing", "Malware", "Access"],
    }
    resolution_codes = ["Solved Remotely", "Solved via On-Site", "Workaround Provided", "Not Reproducible"]

   # Base dates
    opened_dates = pd.date_range("2025-07-01", periods=n, freq="h")

    # Random statuses
    status_selected = np.random.choice(statuses, n, p=[0.3, 0.3, 0.2, 0.2])  # 60% still open/in progress

    closed_dates = []
    resolution_codes_selected = []
    resolved_by = []

    for i, status in enumerate(status_selected):
        if status in ["Resolved", "Closed"]:
            closed_dt = opened_dates[i] + timedelta(hours=random.randint(1, 72))
            closed_dates.append(closed_dt)
            resolution_codes_selected.append(random.choice(resolution_codes))
            resolved_by.append(f"user{random.randint(401,600)}")
        else:
            closed_dates.append(pd.NaT)  # no close date
            resolution_codes_selected.append(None)
            resolved_by.append(None)

    categories_selected = np.random.choice(categories, n)
    subcategories_selected = [random.choice(subcategories[cat]) for cat in categories_selected]

    data = {
        "Incident_ID": [f"INC{i:06d}" for i in range(1, n+1)],
        "Opened_At": opened_dates,
        "Closed_At": closed_dates,
        "Opened_By": [f"user{random.randint(1,200)}" for _ in range(n)],
        "Assigned_To": [f"user{random.randint(201,400)}" for _ in range(n)],
        "Priority": np.random.choice(priorities, n),
        "Impact": np.random.choice(impacts, n),
        "Urgency": np.random.choice(urgencies, n),
        "Category": categories_selected,
        "Subcategory": subcategories_selected,
        "Status": status_selected,
        "Resolution_Code": resolution_codes_selected,
        "Resolved_By": resolved_by,
        "Short_Description": [f"Issue {i} - {random.choice(categories)} related" for i in range(1, n+1)],
        "Description": [f"Detailed description of incident {i}, auto-generated for testing." for i in range(1, n+1)],
        "Updated_At": [d + timedelta(hours=random.randint(1,100)) for d in opened_dates],
    }

    return pd.DataFrame(data)
def chatbot_fn(message, history):
    history = history or []

    if "create csv" in message.lower():
        df = generate_servicenow_data(10000)
        tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
        csv_path = "servicenow_dataset.csv"
        df.to_csv(tmp_file.name, index=False)
    
        # reply: preview + file download
        preview = df.head().to_markdown()
        bot_reply = f"Here’s a preview of the dataset:\n\n{preview}"

        history.append((message, bot_reply))
        history.append(("", (csv_path, "📂 Download ServiceNow CSV (10k rows)")))

        return history, "" 
    
    result = conversation_chain.invoke({"question": message})
    history.append((message,  result["answer"]))
    return history, ""  # clear input


# Gradio UI
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="ServiceNow Chatbot", height=500)
    msg = gr.Textbox(
        label="Type your message here",
        placeholder="Ask me something or type 'create csv'...",
    )

    msg.submit(chatbot_fn, [msg, chatbot], [chatbot, msg])

C:\Users\hp\AppData\Local\Temp\ipykernel_18772\918136923.py:88: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="ServiceNow Chatbot", height=500)


In [14]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
39
IIMA HR Policy Manual 2023
DECLARATION
I, hereby declare that the information provided by me to the 
institute in my job application and thereafter is correct to the best 
of my knowledge and belief. If anything found incorrect or has 
any discrepancies, the institute has the right to terminate my 
appointment at any given time without assigning any reason.  I 
also know that providing wrong/fake information or certificate to 
the institute is punishable under court of law.
        
(Name & Signature) 
Place: Ahmedabad
Date:

40
IIMA HR Policy Manual 2023
JOINING REPORT
To 
Associate Vice President-HR 
IIM Ahmedabad
Through: 
Dear Sir
Thank you

In [ ]:
# def chat(question, history):
#     result = conversation_chain.invoke({"question": question})
#     return result["answer"]

In [ ]:
# # view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

# prompts = [
#     {"role": "system", "content": "You are a helpful assistant"},
#     {"role": "user", "content": "Can you please create a test Dataset for me for Service now Incidents with few open and closed incidents and count ot be 10000 records"}
#   ]
# system_message = "You are a helpful assistant"

# def message_gpt(prompt):
#     messages = [
#         {"role": "system", "content": system_message},
#         {"role": "user", "content": prompt}
#       ]
#     completion = openai.chat.completions.create(
#         model='gpt-4o-mini',
#         messages=messages,
#     )
#     csv_string =  completion.choices[0].message.content
#     print (csv_string)
#     return csv_string

# def process_and_display_csv(prompt):
#         csv_data = message_gpt(prompt)
#         # Convert the CSV string to a DataFrame for better display or further processing
#         try:
#             df = pd.read_csv(io.StringIO(csv_data))
#             return df
#         except Exception as e:
#             return f"Error processing CSV: {e}\nRaw CSV data:\n{csv_data}"

# iface = gr.Interface(
#     fn=process_and_display_csv,
#     inputs=gr.Textbox(lines=5, label="Enter your prompt for CSV generation:"),
#     outputs=gr.Dataframe(label="Generated CSV Data"), # Use gr.Dataframe to display tabular data
#     title="GPT CSV Generator"
# )

# iface.launch(share=False) # Set share=True to get a public link if needed

# #gr.Interface(fn=message_gpt, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

# import gradio as gr
# import openai
# import pandas as pd
# import numpy as np
# import io
# import random
# import tempfile
# from datetime import timedelta

# def chatbot_fn(message, history):
#     history = history or []

#     # special command: create dataset
#     if "create csv" in message.lower():
#         df = generate_servicenow_data(10000)
#         csv_path = "servicenow_dataset.csv"
#         df.to_csv(csv_path, index=False)
#         history.append((message, (csv_path, "📂 Download ServiceNow CSV (10k rows)")))
#         return history, ""  # clear input

#     # otherwise, ask GPT
#     completion = openai.chat.completions.create(
#         model="gpt-4o-mini",  # or "gpt-4o", adjust if needed
#         messages=[{"role": "system", "content": "You are a helpful ServiceNow assistant."}] +
#                  [{"role": "user", "content": m[0]} for m in history if isinstance(m[0], str)] +
#                  [{"role": "assistant", "content": m[1]} for m in history if isinstance(m[1], str)] +
#                  [{"role": "user", "content": message}]
#     )
#     reply = completion.choices[0].message.content
#     history.append((message, reply))
#     return history, ""  # clear input

# def generate_servicenow_data(n=10000):
#     np.random.seed(42)
#     random.seed(42)

#     priorities = ["Low", "Medium", "High", "Critical"]
#     impacts = ["Low", "Medium", "High"]
#     urgencies = ["Low", "Medium", "High"]
#     statuses = ["Open", "In Progress", "Resolved", "Closed"]
#     categories = ["Network", "Software", "Hardware", "Database", "Security"]
#     subcategories = {
#         "Network": ["VPN", "LAN", "WAN"],
#         "Software": ["Email", "OS", "Application"],
#         "Hardware": ["Laptop", "Desktop", "Printer"],
#         "Database": ["Oracle", "MySQL", "SQL Server"],
#         "Security": ["Phishing", "Malware", "Access"],
#     }
#     resolution_codes = ["Solved Remotely", "Solved via On-Site", "Workaround Provided", "Not Reproducible"]

#    # Base dates
#     opened_dates = pd.date_range("2025-07-01", periods=n, freq="h")

#     # Random statuses
#     status_selected = np.random.choice(statuses, n, p=[0.3, 0.3, 0.2, 0.2])  # 60% still open/in progress

#     closed_dates = []
#     resolution_codes_selected = []
#     resolved_by = []

#     for i, status in enumerate(status_selected):
#         if status in ["Resolved", "Closed"]:
#             closed_dt = opened_dates[i] + timedelta(hours=random.randint(1, 72))
#             closed_dates.append(closed_dt)
#             resolution_codes_selected.append(random.choice(resolution_codes))
#             resolved_by.append(f"user{random.randint(401,600)}")
#         else:
#             closed_dates.append(pd.NaT)  # no close date
#             resolution_codes_selected.append(None)
#             resolved_by.append(None)

#     categories_selected = np.random.choice(categories, n)
#     subcategories_selected = [random.choice(subcategories[cat]) for cat in categories_selected]

#     data = {
#         "Incident_ID": [f"INC{i:06d}" for i in range(1, n+1)],
#         "Opened_At": opened_dates,
#         "Closed_At": closed_dates,
#         "Opened_By": [f"user{random.randint(1,200)}" for _ in range(n)],
#         "Assigned_To": [f"user{random.randint(201,400)}" for _ in range(n)],
#         "Priority": np.random.choice(priorities, n),
#         "Impact": np.random.choice(impacts, n),
#         "Urgency": np.random.choice(urgencies, n),
#         "Category": categories_selected,
#         "Subcategory": subcategories_selected,
#         "Status": status_selected,
#         "Resolution_Code": resolution_codes_selected,
#         "Resolved_By": resolved_by,
#         "Short_Description": [f"Issue {i} - {random.choice(categories)} related" for i in range(1, n+1)],
#         "Description": [f"Detailed description of incident {i}, auto-generated for testing." for i in range(1, n+1)],
#         "Updated_At": [d + timedelta(hours=random.randint(1,100)) for d in opened_dates],
#     }

#     return pd.DataFrame(data)
# def chatbot_fn(message, history):
#     history = history or []

#     if "create csv" in message.lower():
#         df = generate_servicenow_data(10000)
#         tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
#         csv_path = "servicenow_dataset.csv"
#         df.to_csv(tmp_file.name, index=False)
    
#         # reply: preview + file download
#         preview = df.head().to_markdown()
#         bot_reply = f"Here’s a preview of the dataset:\n\n{preview}"

#         history.append((message, bot_reply))
#         history.append(("", (csv_path, "📂 Download ServiceNow CSV (10k rows)")))

#         return history, ""  # clear input


#     # otherwise, ask GPT
#     completion = openai.chat.completions.create(
#         model="gpt-4o-mini",  # or "gpt-4o", adjust if needed
#         messages=[{"role": "system", "content": "You are a helpful ServiceNow assistant."}] +
#                  [{"role": "user", "content": m[0]} for m in history if isinstance(m[0], str)] +
#                  [{"role": "assistant", "content": m[1]} for m in history if isinstance(m[1], str)] +
#                  [{"role": "user", "content": message}]
#     )
#     reply = completion.choices[0].message.content
#     history.append((message, reply))
#     return history, ""  # clear input


# # Gradio UI
# with gr.Blocks() as demo:
#     chatbot = gr.Chatbot(label="ServiceNow Chatbot", height=500)
#     msg = gr.Textbox(
#         label="Type your message here",
#         placeholder="Ask me something or type 'create csv'...",
#     )

#     msg.submit(chatbot_fn, [msg, chatbot], [chatbot, msg])

# demo.launch()